# Introduction/Business Problem section 
### A description of the problem and a discussion of the background

In this project we will try to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening an **Asian restaurant** in **London**, UK.

Since there are lots of restaurants in LONDON we will try to detect **locations that are not already crowded with restaurants**. We are also particularly interested in **areas with no Asian restaurants in vicinity**. We would also prefer locations **as close to city center as possible**, assuming that first two conditions are met.

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

# Data section 

### A description of the data and how it will be used to solve the problem.

Based on definition of our problem, factors that will influence our decission are:
* number of existing restaurants in the neighborhood (any type of restaurant)
* number of and distance to Asian restaurants in the neighborhood, if any
* distance of neighborhood from city center

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **Google Maps API reverse geocoding**
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**
* coordinate of London center will be obtained using **Google Maps API geocoding** of well known London location(Southbank,London)

In [63]:
google_api_key = ""

In [6]:
import requests

def get_coordinates(api_key, address, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(api_key, address)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        geographical_data = results[0]['geometry']['location'] # get geographical coordinates
        lat = geographical_data['lat']
        lon = geographical_data['lng']
        return [lat, lon]
    except:
        return [None, None]
    
address = 'South Bank, London, UK'
London_center = get_coordinates(google_api_key, address)
print('Coordinate of {}: {}'.format(address, London_center))

Coordinate of South Bank, London, UK: [51.503728, -0.1116764]


Now let's create a grid of area candidates, equaly spaced, centered around city center and within ~6km from South Bank,London. Our neighborhoods will be defined as circular areas with a radius of 300 meters, so our neighborhood centers will be 600 meters apart.

To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in meters).

In [36]:
import shapely.geometry
from pyproj import Transformer,CRS
import math


def lonlat_to_xy(lon, lat):
    #proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    #proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    
    #xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    crs_4326 = CRS("EPSG:4326")
    crs_proj = CRS("EPSG:32633")
    transformer = Transformer.from_crs(crs_4326,crs_proj)
    xy = transformer.transform(lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    #proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    #proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    #lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    crs_4326 = CRS("EPSG:4326")
    crs_proj = CRS("EPSG:32633")
    transformer = Transformer.from_crs(crs_proj,crs_4326)
    lonlat = transformer.transform(x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('London center longitude={}, latitude={}'.format(London_center[1], London_center[0]))
x, y = lonlat_to_xy(London_center[1], London_center[0])
print('London center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('London center longitude={}, latitude={}'.format(lo, la))

Coordinate transformation check
-------------------------------
London center longitude=-0.1116764, latitude=51.503728
London center UTM X=4870933.351789917, Y=-15384.6512124353
London center longitude=-0.11167639999999977, latitude=51.503727999999995


Let's create a hexagonal grid of cells: we offset every other row, and adjust vertical row spacing so that every cell center is equally distant from all it's neighbors.

In [37]:
London_center_x, London_center_y = lonlat_to_xy(London_center[1], London_center[0]) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = London_center_x - 6000
x_step = 600
y_min = London_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(London_center_x, London_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

364 candidate neighborhood centers generated.


Let's visualize the data we have so far: city center location and candidate neighborhood centers:

In [27]:
import folium

In [38]:
map_london = folium.Map(location=London_center, zoom_start=13)
folium.Marker(London_center, popup='South Bank').add_to(map_london)
for lat, lon in zip(latitudes, longitudes):

    folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_london)

map_london

We now have the coordinates of centers of neighborhoods/areas to be evaluated, equally spaced (distance from every point to it's neighbors is exactly the same) and within ~6km from South Bank, London.

Let's now use Google Maps API to get approximate addresses of those locations.

In [39]:
def get_address(api_key, latitude, longitude, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&latlng={},{}'.format(api_key, latitude, longitude)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        address = results[0]['formatted_address']
        return address
    except:
        return None

addr = get_address(google_api_key, London_center[0], London_center[1])
print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(London_center[0], London_center[1], addr))

Reverse geocoding check
-----------------------
Address of [51.503728, -0.1116764] is: Waterloo Station Waterloo Road (Stop F), South Bank, London SE1 8SG, UK


In [40]:
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = get_address(google_api_key, lat, lon)
    if address is None:
        address = 'NO ADDRESS'
    address = address.replace(', UK', '') # We don't need country part of address
    addresses.append(address)
    print(' .', end='')
print(' done.')

Obtaining location addresses:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [42]:
addresses[0:5]

['40 Pimlico Rd, Belgravia, London SW1W 8LP',
 '12 Lyall St, Belgravia, London SW1X 8DH',
 '45 Belgrave Square, Belgravia, London SW1X 8PA',
 'S Carriage Dr, London W1J 7NT',
 '2 Park St, Mayfair, London W1K 2XA']

Looking good. Let's now place all this into a Pandas dataframe.

In [43]:
import pandas as pd

df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center
0,"40 Pimlico Rd, Belgravia, London SW1W 8LP",51.490819,-0.153193,4.869133e+06,-21100.418877,5992.495307
1,"12 Lyall St, Belgravia, London SW1X 8DH",51.495146,-0.153184,4.869733e+06,-21100.418877,5840.376700
2,"45 Belgrave Square, Belgravia, London SW1X 8PA",51.499472,-0.153175,4.870333e+06,-21100.418877,5747.173218
3,"S Carriage Dr, London W1J 7NT",51.503799,-0.153167,4.870933e+06,-21100.418877,5715.767665
4,"2 Park St, Mayfair, London W1K 2XA",51.508125,-0.153158,4.871533e+06,-21100.418877,5747.173218
5,"40 N Audley St, Mayfair, London W1K 6WG",51.512451,-0.153150,4.872133e+06,-21100.418877,5840.376700
6,"Ici plc, 20 Manchester Square, Marylebone, Lon...",51.516776,-0.153141,4.872733e+06,-21100.418877,5992.495307
7,"8 Chelsea Bridge, London SW8 4NW",51.484321,-0.149433,4.868233e+06,-20580.803635,5855.766389
8,"Doneraile House, Ebury Bridge Rd, London SW1W 8SX",51.488648,-0.149424,4.868833e+06,-20580.803635,5604.462508
9,"19 Elizabeth St, Belgravia, London SW1W 9RP",51.492975,-0.149416,4.869433e+06,-20580.803635,5408.326913


...and let's now save/persist this data into local file.

In [49]:
df_locations.to_pickle('./locations.pkl')    

### Foursquare
Now that we have our location candidates, let's use Foursquare API to get info on restaurants in each neighborhood.

We're interested in venues in 'food' category, but only those that are proper restaurants - coffe shops, pizza places, bakeries etc. are not direct competitors so we don't care about those. So we will include in out list only venues that have 'restaurant' in category name, and we'll make sure to detect and include all the subcategories of specific 'Asian restaurant' category, as we need info on Asian restaurants in the neighborhood.

In [64]:
client_id = '' # your Foursquare ID
client_secret = '' # your Foursquare Secret
version = '20180604'

In [53]:
# Category IDs corresponding to Asian restaurants were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):

food_category = '4d4b7105d754a06374d81259' # 'Root' category for all food-related venues

asian_restaurant_categories = ['4bf58dd8d48988d142941735','56aa371be4b08b9a8d573568','52e81612bcbc57f1066b7a03',
                                 '4bf58dd8d48988d145941735','4eb1bd1c3b7b55596b4a748f','52e81612bcbc57f1066b79fb',
                                 '4deefc054765f83613cdba6f','4bf58dd8d48988d111941735','4bf58dd8d48988d113941735',
                                 '4bf58dd8d48988d156941735','4eb1d5724b900d56c88a45fe','4bf58dd8d48988d149941735',
                                 '52af39fb3cf9994f4e043be9','4bf58dd8d48988d14a941735']

def is_restaurant(categories, specific_filter=None):
    restaurant_words = ['restaurant', 'diner', 'taverna', 'steakhouse']
    restaurant = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in restaurant_words:
            if r in category_name:
                restaurant = True
        if 'fast food' in category_name:
            restaurant = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            restaurant = True
    return restaurant, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
   
    address = address.replace(', UK', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [54]:
# Let's now go over our neighborhood locations and get nearby restaurants; we'll also maintain a dictionary of all found restaurants and all found Asian restaurants

import pickle

def get_restaurants(lats, lons):
    restaurants = {}
    asian_restaurants = {}
    location_restaurants = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, client_id, client_secret, radius=350, limit=100)
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_asian = is_restaurant(venue_categories, specific_filter=asian_restaurant_categories)
            if is_res:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_asian, x, y)
                if venue_distance<=300:
                    area_restaurants.append(restaurant)
                restaurants[venue_id] = restaurant
                if is_asian:
                    asian_restaurants[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
        print(' .', end='')
    print(' done.')
    return restaurants, asian_restaurants, location_restaurants

# Try to load from local file system in case we did this before
restaurants = {}
asian_restaurants = {}
location_restaurants = []
loaded = False
try:
    with open('restaurants_350.pkl', 'rb') as f:
        restaurants = pickle.load(f)
    with open('asian_restaurants_350.pkl', 'rb') as f:
        asian_restaurants = pickle.load(f)
    with open('location_restaurants_350.pkl', 'rb') as f:
        location_restaurants = pickle.load(f)
    print('Restaurant data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    restaurants, asian_restaurants, location_restaurants = get_restaurants(latitudes, longitudes)
    
    # Let's persists this in local file system
    with open('restaurants_350.pkl', 'wb') as f:
        pickle.dump(restaurants, f)
    with open('Asian_restaurants_350.pkl', 'wb') as f:
        pickle.dump(asian_restaurants, f)
    with open('location_restaurants_350.pkl', 'wb') as f:
        pickle.dump(location_restaurants, f)
        

Obtaining venues around candidate locations: . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [56]:
import numpy as np

print('Total number of restaurants:', len(restaurants))
print('Total number of Asian restaurants:', len(asian_restaurants))
print('Percentage of Asian restaurants: {:.2f}%'.format(len(asian_restaurants) / len(restaurants) * 100))
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())

Total number of restaurants: 2017
Total number of Asian restaurants: 373
Percentage of Asian restaurants: 18.49%
Average number of restaurants in neighborhood: 9.934065934065934


In [57]:
print('List of all restaurants')
print('-----------------------')
for r in list(restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(restaurants))

List of all restaurants
-----------------------
('4ac518d4f964a520bca720e3', 'La Poule au Pot', 51.49086426038942, -0.15270073351852484, '231 Ebury St, London, Greater London, SW1W 8UT, United Kingdom', 327, False, 4869139.763431154, -21032.66048564754)
('4c40750e7beab71394519555', 'Enoteca Turi', 51.49024191037212, -0.15436312373679156, '87 Pimlico Rd, London, Greater London, SW1W 8PH, United Kingdom', 103, False, 4869053.009946184, -21261.46120522153)
('4ac518e6f964a52010ab20e3', 'Hunan', 51.490698490413905, -0.1528200452064199, '51 Pimlico Rd (at Ebury St), London, Greater London, SW1W 8NE, United Kingdom', 346, True, 4869116.744396659, -21049.048649822686)
('506ac702e4b0dcfc950fce65', 'Colbert', 51.492818, -0.156647, '50-52 Sloane Sq, Chelsea, Greater London, SW1W 8AX, United Kingdom', 326, False, 4869409.593027143, -21576.75898486836)
('4ad0b37ef964a52035d920e3', 'The Botanist', 51.49273166187537, -0.15694219815772975, '7 Sloane Sq, Kensington, Greater London, SW1W 8EE, United Kin

In [58]:
print('List of Asian restaurants')
print('---------------------------')
for r in list(asian_restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(asian_restaurants))

List of Asian restaurants
---------------------------
('4ac518e6f964a52010ab20e3', 'Hunan', 51.490698490413905, -0.1528200452064199, '51 Pimlico Rd (at Ebury St), London, Greater London, SW1W 8NE, United Kingdom', 346, True, 4869116.744396659, -21049.048649822686)
('4dab3ff95da3ba8a479d5211', "Ken Lo's Memories Of China", 51.49485074649806, -0.1483486608671512, '67 Ebury St., London, Greater London, SW1 W 8, United Kingdom', 190, True, 4869693.7342306655, -20434.2799066966)
('4ac518d6f964a52050a820e3', 'Nobu', 51.50506856589684, -0.1505569830427778, '19 Old Park Ln, London, Greater London, W1K 1LB, United Kingdom', 127, True, 4871110.169546546, -20741.2299319929)
('4ac518dcf964a5204aa920e3', 'Kai', 51.50882279965133, -0.1514459399898466, '65 South Audley St, London, Greater London, W1K 2QU, United Kingdom', 216, True, 4871630.620714631, -20864.71775332782)
('4ac518e5f964a520fdaa20e3', 'China Tang', 51.50733588323953, -0.15303006098065366, 'The Dorchester, 53 Park Lane, London, Greater 

In [59]:
print('Restaurants around location')
print('---------------------------')
for i in range(100, 110):
    rs = location_restaurants[i][:8]
    names = ', '.join([r[1] for r in rs])
    print('Restaurants around location {}: {}'.format(i+1, names))

Restaurants around location
---------------------------
Restaurants around location 101: Pop Art Sushi!, Waterfront London Vauxhall Restaurant, Harare Restaurant, Mumbai Delight, STEAX AND THE CITY, Cottons Rum Shack & Restaurant, Tony's Delicatessen
Restaurants around location 102: Casa Madeira, Waterfront London Vauxhall Restaurant, Rex Whistler, STEAX AND THE CITY, Pico Bar & Grill
Restaurants around location 103: Shepherd's of Westminster, Firecracker, City Cafe, Rex Whistler, Art St Kitchen
Restaurants around location 104: The Cinnamon Club, itsu, Quirinale, Osteria Dell'Angolo, Ma La Sichuan, Firecracker, Atrium
Restaurants around location 105: Roux at Parliament Square, The Adjournment, Members Dining Room
Restaurants around location 106: Kerridge’s Bar & Grill, The Northall, Thai Square, Rochelle Canteen at ICA, The Whitehall, One Twenty One Two, Blue Garden, Walkers of Whitehall
Restaurants around location 107: Flat Iron, Barrafina, CoCo Ichibanya, Tandoor Chop House, J Sheeke

Let's now see all the collected restaurants in our area of interest on map, and let's also show asian restaurants in different color.

In [62]:
map_london = folium.Map(location=London_center, zoom_start=13)
folium.Marker(London_center, popup='South Bank').add_to(map_london)
for res in restaurants.values():
    lat = res[2]; lon = res[3]
    is_asian = res[6]
    color = 'red' if is_asian else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_london)
map_london